# Implementation of the Algorithm Using Torch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch

In [ ]:
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')
ctgan = pd.read_csv('Syn data/ctgan_5000.csv')
gauscop = pd.read_csv('Syn data/gauscop_5000.csv')
synthpop = pd.read_csv('Syn data/synthpop_5000.csv')
tvae = pd.read_csv('Syn data/tvae_5000.csv')
syndata = pd.read_csv('synData9.csv')
realdata34 = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

realdata34.drop('STDs: Time since first diagnosis', axis=1, inplace=True)
realdata34.drop('STDs: Time since last diagnosis', axis=1, inplace=True)

In [ ]:
med34 = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis']

med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']

bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

In [ ]:
def impute(data, graph=False):
    temp_data = data
    temp_data[temp_data == '?'] = np.nan

    for i in temp_data.columns:
        if(temp_data[i].dtype == 'O'):
            temp_data[i] = temp_data[i].astype('float')
    if len(data.columns) == 34:
        med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis']
    else:
        med = ['Age', 'Number of sexual partners', 'First sexual intercourse',
        'Num of pregnancies', 'Smokes (years)','Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)',
        'STDs (number)', 'STDs: Number of diagnosis',
        'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']
        
    for i in med:
        imp = float(temp_data[i].median())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    bin = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'STDs', 'STDs:condylomatosis',
        'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
        'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
        'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
        'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
        'STDs:Hepatitis B', 'STDs:HPV','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
        'Citology', 'Biopsy']

    for i in bin:
        imp = float(temp_data[i].mode())
        temp_data[i].fillna(imp, inplace=True)
        temp_data[i].replace(np.nan, imp)

    if graph:
        temp_data.hist(figsize=(25,15))
    
    return temp_data

def norm(data1):
    """
    Min-Max normalizes the continuous data
    """
    new_dat = data1.copy()
    for col in new_dat.columns:
        if col in med:
            denom = new_dat[col].max() - new_dat[col].min()
            if denom != 0:
                new_dat[col] = (new_dat[col] - new_dat[col].min()) / denom
            else:
                new_dat[col] = 0
    return new_dat

In [ ]:
realdataA = norm(impute(realdata))
syndataA = norm(impute(syndata))
ctganA = norm(impute(ctgan))
gauscopA = norm(impute(gauscop))
synthpopA = norm(impute(synthpop))
tvaeA = norm(impute(tvae))
realdata34A = norm(impute(realdata34))

syndata.isnull().values.any(), realdata34.isnull().values.any(), realdata.isnull().values.any(), ctganA.isnull().values.any(), gauscopA.isnull().values.any(), synthpopA.isnull().values.any(), tvaeA.isnull().values.any()

In [ ]:
def compute_distance(df1, df2):
    
    ar1 = df1.to_numpy()
    ar2 = df2.to_numpy()
    x1 = torch.Tensor(ar1)
    x2 = torch.Tensor(ar2)

    y1 = x1.unsqueeze(2).expand(x1.size()[0], x1.size()[1], x2.size()[0])

    y2 = x2.unsqueeze(2).expand(x2.size()[0], x2.size()[1], x1.size()[0]).permute(2,1,0)

 
    weights = torch.ones(x1.size()[1])
    
    Δy = torch.abs(y1-y2)
    result = torch.matmul(Δy.permute(0,2,1), weights).numpy()
    return result

In [ ]:
syndataB = compute_distance(realdataA, syndataA)
ctganB = compute_distance(realdata34A, ctganA)
gauscopB = compute_distance(realdata34A, gauscopA)
synthpopB = compute_distance(realdata34A, synthpopA)
tvaeB = compute_distance(realdata34A, tvaeA)
realvreal = compute_distance(realdataA, realdataA)
real34vreal34 = compute_distance(realdata34A, realdata34A)

In [ ]:
newSyn = []
newCt = []
newGaus = []
newSynth = []
newTvae = []

for i in range(858):
    newSyn.append(syndataB[i][:858].tolist())
    newCt.append(ctganB[i][:858].tolist())
    newGaus.append(gauscopB[i][:858].tolist())
    newSynth.append(synthpopB[i][:858].tolist())
    newTvae.append(tvaeB[i][:858].tolist())  

In [ ]:
plt.imshow(real34vreal34[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(realvreal[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(syndataB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(ctganB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(gauscopB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(synthpopB[:,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
plt.imshow(tvaeB[:858,:858], cmap=mpl.cm.get_cmap('cividis_r'))
plt.colorbar()

In [ ]:
def rank(ar):
    """
    Takes in an array, ranks it and returns it
    """
    old_ar = ar
    sorted_ar = sorted(ar)
    for i in range(len(ar)):
        ind_in_old = old_ar.index(sorted_ar[i])
        old_ar[ind_in_old] = i


    return old_ar

def synrank(ar):
    """
    The ranking algo with changing shape of arrays
    """
    new_ar = []
    for j in range(len(ar[0])):
        curr_ar = []
        for i in range(len(ar)):        
            curr_ar.append(ar[i][j])
        new_ar.append(curr_ar)

    
    return new_ar

In [ ]:
syndataB = compute_distance(realdataA, syndataA)
ctganB = compute_distance(realdata34A, ctganA)
gauscopB = compute_distance(realdata34A, gauscopA)
synthpopB = compute_distance(realdata34A, synthpopA)
tvaeB = compute_distance(realdata34A, tvaeA)
realvreal = compute_distance(realdataA, realdataA)
real34vreal34 = compute_distance(realdata34A, realdata34A)

In [ ]:
newSyn = []
newCt = []
newGaus = []
newSynth = []
newTvae = []

In [ ]:
dictSyn = {}
dictCT = {}
dictGaus = {}
dictSynth = {}
dictTvae = {}
dictSynR = {}
dictCTR = {}
dictGausR = {}
dictSynthR = {}
dictTvaeR = {}


keys = range(858)
for i in keys:
    dictSyn[i] = rank(synrank(newSyn[i]))
    dictCT[i] = rank(synrank(newCt[i]))
    dictGaus[i] = rank(synrank(newGaus[i]))
    dictSynth[i] = rank(synrank(newSynth[i]))
    dictTvae[i] = rank(synrank(newTvae[i]))
    dictSynR[i] = rank(newSyn[i])
    dictCTR[i] = rank(newCt[i])
    dictGausR[i] = rank(newGaus[i])
    dictSynthR[i] = rank(newSynth[i])
    dictTvaeR[i] = rank(newTvae[i])

In [ ]:
from matching.algorithms import galeshapley
matching = galeshapley(dictS, dictR)
matching

In [ ]:
#dictS
A = torch.Tensor([[1,3,4,0],[3,7,5,9]])

In [ ]:
s = torch.argsort(A, dim=1)
len(A)

In [ ]:
#type(dictS[0])
def createDict(A):
    s = torch.argsort(torch.Tensor(A),dim=1)
    d = {}
    for i in range(len(A)):
        d[i] = s[i,:].tolist()
    return d

In [ ]:
#synD, realD
A = compute_distance(realdataC, syndataC[:858])
dictR = createDict(A)
dictS = createDict(A.T)
#matching = galeshapley(dictS,dictR)
#matching

In [ ]:


#Plot the matching
#Will need to convert dict into nnumpy array
#Then plot it
#Do for all synthetic datasets
#And the real vs real

Create the graphs in Notion